In [198]:
from WebTable import findTables, pullTable
from bs4 import BeautifulSoup
import requests
import re
import bs4
import pandas as pd
import datetime

In [199]:
findTables('https://www.pro-football-reference.com/teams/tam/2018.htm')

['team_stats',
 'games',
 'team_conversions',
 'passing',
 'rushing_and_receiving',
 'returns',
 'kicking',
 'defense',
 'scoring',
 'team_td_log',
 'opp_td_log']

In [200]:
def team_rename(team_s):
    try:
        if team_s == 'Arizona Cardinals':
            ts = 'crd'
        elif team_s == 'Atlanta Falcons':
            ts = 'atl'
        elif team_s == 'Baltimore Ravens':
            ts = 'rav'
        elif team_s == 'Buffalo Bills':
            ts = 'buf'
        elif team_s == 'Carolina Panthers':
            ts = 'car'
        elif team_s == 'Chicago Bears':
            ts = 'chi'
        elif team_s == 'Cincinnati Bengals':
            ts = 'cin'
        elif team_s == 'Cleveland Browns':
            ts = 'cle'
        elif team_s == 'Dallas Cowboys':
            ts = 'dal'
        elif team_s == 'Denver Broncos':
            ts = 'den'
        elif team_s == 'Detroit Lions':
            ts = 'det'
        elif team_s == 'Green Bay Packers':
            ts = 'gnb'
        elif team_s == 'Houston Texans':
            ts = 'htx'
        elif team_s == 'Indianapolis Colts':
            ts = 'clt'
        elif team_s == 'Jacksonville Jaguars':
            ts = 'jax'
        elif team_s == 'Kansas City Chiefs':
            ts = 'kan'
        elif team_s == 'Miami Dolphins':
            ts = 'mia'
        elif team_s == 'Minnesota Vikings':
            ts = 'min'
        elif team_s == 'New England Patriots':
            ts = 'nwe'
        elif team_s == 'New Orleans Saints':
            ts = 'nor'
        elif team_s == 'New York Giants':
            ts = 'nyg'
        elif team_s == 'New York Jets':
            ts = 'nyj'
        elif team_s == 'Oakland Raiders':
            ts = 'rai'
        elif team_s == 'Philadelphia Eagles':
            ts = 'phi'
        elif team_s == 'Pittsburgh Steelers':
            ts = 'pit'
        elif team_s == 'San Diego Chargers':
            ts = 'sdg'
        elif team_s == 'San Francisco 49ers':
            ts = 'sfo'
        elif team_s == 'Seattle Seahawks':
            ts = 'sea'
        elif team_s == 'St. Louis Rams':
            ts = 'ram'
        elif team_s == 'Tampa Bay Buccaneers':
            ts = 'tam'
        elif team_s == 'Tennessee Titans':
            ts = 'oti'
        elif team_s == 'Washington Redskins':
            ts = 'was'
    except NameError:
        print('Invalid team name')
    return ts

In [201]:
team_rename('Washington Redskins')

'was'

In [299]:
team_list = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det','gnb', 
'htx', 'clt', 'jax', 'kan', 'mia', 'min', 'nwe', 'nor', 'nyg', 'nyj', 'rai', 'phi', 'pit', 
'sdg', 'sfo', 'sea', 'ram', 'tam', 'oti', 'was']
# team_list = ['rai']
year = str(2019)
nflstatsdict = {}
for team in team_list:
    base = 'https://www.pro-football-reference.com'
    url = base + '/teams/' + team + '/' + year + '.htm'
    result = requests.get('https://www.pro-football-reference.com/teams/' + team + '/' + year + '.htm')
    soup = bs4.BeautifulSoup(result.text)
    table_gamelog = soup.find('div', id = 'div_games')
    dict1 = {}
    teamYearStats = {}
    ordered_keys = []
    for count, row in enumerate(table_gamelog.find_all("th")[5:]):
        if count in [0,1,2]:
            dict1[row.text] = []
            ordered_keys.append(row.text)
        elif count == 3:
            dict1['Time'] = []
            ordered_keys.append('Time')
        elif count == 4:
            dict1['Boxscore'] = []
            ordered_keys.append('Boxscore')
        elif count == 5:
            dict1['W/L'] = []
            ordered_keys.append('W/L')
        elif count in [6,7]:
            dict1[row.text] = []
            ordered_keys.append(row.text)
        elif count == 8:
            dict1['Home'] = []
            ordered_keys.append('Home')
        elif count == 9:
            dict1['Opp_Name'] = []
            ordered_keys.append('Opp_Name')
        elif count in [x for x in range(10,17)]:
            dict1[row.text] = []
            ordered_keys.append(row.text)
        elif count in [x for x in range(17,22)]:
            name = 'Def_'
            dict1[name + row.text] = []
            ordered_keys.append(name + row.text)
        elif count in [x for x in range(22,25)]:
            name = 'ExpectedPoints_'
            dict1[name + row.text] = []
            ordered_keys.append(name +row.text)
        elif count in [x for x in range(25,43)]:
            week = 'Week_'
            dict1[week + row.text] = []
            ordered_keys.append(week + row.text)
    for count1, row in enumerate(table_gamelog.find_all('tr')):
        for count, x in enumerate(row.find_all('td')):
#             print(count +1 ,x.text)
            if count in range(0,3):
                dict1[ordered_keys[count + 1]].append(x.text)
            elif count == 3:                                        #link to Boxscore stats
                for a in x.find_all('a', href=True):
                    a = a['href']
                    if a.startswith('/boxscores'):
                        statslink = a
                        statslink = base + statslink
                        dict1['Boxscore'] = boxscore(statslink)
                dict1[ordered_keys[count + 1]].append(statslink)
            elif count in range(5,16):                              # Offense Stats
                dict1[ordered_keys[count + 1]].append(x.text)
            elif count in range(16,21):                             # Defense stats
                dict1[ordered_keys[count + 1]].append(x.text)
    teamYearStats[team + ' ' +year] = dict1
    nflstatsdict.update(teamYearStats)
    
            

In [300]:
nflstatsdict

{'crd 2019': {'Week': [],
  'Day': ['Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Thu',
   'Sun',
   'Sun',
   '',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun'],
  'Date': ['September 8',
   'September 15',
   'September 22',
   'September 29',
   'October 6',
   'October 13',
   'October 20',
   'October 27',
   'October 31',
   'November 10',
   'November 17',
   '',
   'December 1',
   'December 8',
   'December 15',
   'December 22',
   'December 29'],
  'Time': ['4:25PM ET',
   '1:00PM ET',
   '4:05PM ET',
   '4:05PM ET',
   '1:00PM ET',
   '4:05PM ET',
   '1:00PM ET',
   '1:00PM ET',
   '8:20PM ET',
   '1:00PM ET',
   '4:05PM ET',
   '',
   '4:05PM ET',
   '4:25PM ET',
   '4:05PM ET',
   '4:25PM ET',
   '4:25PM ET'],
  'Boxscore': [('Home_First Downs', '13'),
   ('Home_Rush-Yds-TDs', '20-88-0'),
   ('Home_Cmp-Att-Yd-TD-INT', '18-37-206-1-1'),
   ('Home_Sacked-Yards', '2-23'),
   ('Home_Net Pass Yards', '183'),
   ('Home_Total Yards', '271'),

In [203]:
#boxscore


###### with open('filename') as f:
    for x in range (number of fields):
        zip(list(nlfstatsdict.values()))

In [204]:
nflstatsdict

{'crd 2019': {'Week': [],
  'Day': ['Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Thu',
   'Sun',
   'Sun',
   '',
   'Sun',
   'Sun',
   'Sun',
   'Sun',
   'Sun'],
  'Date': ['September 8',
   'September 15',
   'September 22',
   'September 29',
   'October 6',
   'October 13',
   'October 20',
   'October 27',
   'October 31',
   'November 10',
   'November 17',
   '',
   'December 1',
   'December 8',
   'December 15',
   'December 22',
   'December 29'],
  'Time': ['4:25PM ET',
   '1:00PM ET',
   '4:05PM ET',
   '4:05PM ET',
   '1:00PM ET',
   '4:05PM ET',
   '1:00PM ET',
   '1:00PM ET',
   '8:20PM ET',
   '1:00PM ET',
   '4:05PM ET',
   '',
   '4:05PM ET',
   '4:25PM ET',
   '4:05PM ET',
   '4:25PM ET',
   '4:25PM ET'],
  'Boxscore': ['https://www.pro-football-reference.com/boxscores/201909080crd.htm',
   'https://www.pro-football-reference.com/boxscores/201909150rav.htm',
   'https://www.pro-football-reference.com/boxscores/201909220crd.htm',


In [196]:
teams = list(nflstatsdict.keys())
col_names = list(nflstatsdict[teams[0]].keys())

nflstatsdict[teams[0]][col_names[2]]

list(zip(nflstatsdict[teams[0]]))

#for team in teams:
 #   for col in col_names:
 #       print(nflstatsdict[team][col])
# nested for loops to zip into csv
#col_names
# for x in test:
#     for line in list(nflstatsdict[x].values()):
#         zipped = zip(x,line)
#         print(list(zipped))
#     print(line)

[('Week',),
 ('Day',),
 ('Date',),
 ('Time',),
 ('Boxscore',),
 ('W/L',),
 ('OT',),
 ('Rec',),
 ('Home',),
 ('Opp_Name',),
 ('Tm',),
 ('Opp',),
 ('1stD',),
 ('TotYd',),
 ('PassY',),
 ('RushY',),
 ('TO',),
 ('Def_1stD',),
 ('Def_TotYd',),
 ('Def_PassY',),
 ('Def_RushY',),
 ('Def_TO',),
 ('ExpectedPoints_Offense',),
 ('ExpectedPoints_Defense',),
 ('ExpectedPoints_Sp. Tms',),
 ('Week_1',),
 ('Week_2',),
 ('Week_3',),
 ('Week_4',),
 ('Week_5',),
 ('Week_6',),
 ('Week_7',),
 ('Week_8',),
 ('Week_9',),
 ('Week_10',),
 ('Week_11',),
 ('Week_12',),
 ('Week_13',),
 ('Week_14',),
 ('Week_15',),
 ('Week_16',),
 ('Week_17',)]

In [152]:
df = pd.DataFrame.from_dict(nflstatsdict, orient='index')
pd.DataFrame.from_dict({(i,j): user_dict[i][j] 
                           for i in user_dict.keys() 
                           for j in user_dict[i].keys()}

,Week,Day,Date,Time,Boxscore,W/L,OT,Rec,Home,Opp_Name,...,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,Week_
crd 2019,[],"[Sun, Sun, Sun, Sun, Sun, Sun, Sun, Sun, Thu, ...","[September 8, September 15, September 22, Sept...","[4:25PM ET, 1:00PM ET, 4:05PM ET, 4:05PM ET, 1...",[https://www.pro-football-reference.com/boxsco...,[],"[OT, , , , , , , , , , , , , , , , ]","[0-0-1, 0-1-1, 0-2-1, 0-3-1, 1-3-1, 2-3-1, 3-3...","[, @, , , @, , @, @, , @, @, , , , , @, @]","[Detroit Lions, Baltimore Ravens, Carolina Pan...",...,[],[],[],[],[],[],[],[],[],NaN
atl 2019,[],"[Sun, Sun, Sun, Sun, Sun, Sun, Sun, Sun, , Sun...","[September 8, September 15, September 22, Sept...","[1:00PM ET, 8:20PM ET, 1:00PM ET, 1:00PM ET, 1...",[https://www.pro-football-reference.com/boxsco...,[],"[, , , , , , , , , , , , , , , , OT]","[0-1, 1-1, 1-2, 1-3, 1-4, 1-5, 1-6, 1-7, , 2-7...","[@, , @, , @, @, , , , @, @, , , , @, , @]","[Minnesota Vikings, Philadelphia Eagles, India...",...,[],[],[],[],[],[],[],[],[],NaN
rav 2019,[],"[Sun, Sun, Sun, Sun, Sun, Sun, Sun, , Sun, Sun...","[September 8, September 15, September 22, Sept...","[1:00PM ET, 1:00PM ET, 1:00PM ET, 1:00PM ET, 1...",[https://www.pro-football-reference.com/boxsco...,[],"[, , , , OT, , , , , , , , , , , , , , ]","[1-0, 2-0, 2-1, 2-2, 3-2, 4-2, 5-2, , 6-2, 7-2...","[@, , @, , @, , @, , , @, , @, , @, , @, , , ]","[Miami Dolphins, Arizona Cardinals, Kansas Cit...",...,[],[],[],[],[],[],[],[],[],[]
buf 2019,[],"[Sun, Sun, Sun, Sun, Sun, , Sun, Sun, Sun, Sun...","[September 8, September 15, September 22, Sept...","[1:00PM ET, 1:00PM ET, 1:00PM ET, 1:00PM ET, 1...",[https://www.pro-football-reference.com/boxsco...,[],"[, , , , , , , , , , , , , , , , , , OT]","[1-0, 2-0, 3-0, 3-1, 4-1, , 5-1, 5-2, 6-2, 6-3...","[@, @, , , @, , , , , @, @, , @, , @, @, , , @]","[New York Jets, New York Giants, Cincinnati Be...",...,[],[],[],[],[],[],[],[],[],[]
car 2019,[],"[Sun, Thu, Sun, Sun, Sun, Sun, , Sun, Sun, Sun...","[September 8, September 12, September 22, Sept...","[1:00PM ET, 8:20PM ET, 4:05PM ET, 1:00PM ET, 1...",[https://www.pro-football-reference.com/boxsco...,[],"[, , , , , , , , , , , , , , , , ]","[0-1, 0-2, 1-2, 2-2, 3-2, 4-2, , 4-3, 5-3, 5-4...","[, , @, @, , @, , @, , @, , @, , @, , @, ]","[Los Angeles Rams, Tampa Bay Buccaneers, Arizo...",...,[],[],[],[],[],[],[],[],[],NaN
chi 2019,[],"[Thu, Sun, Mon, Sun, Sun, , Sun, Sun, Sun, Sun...","[September 5, September 15, September 23, Sept...","[8:20PM ET, 4:25PM ET, 8:15PM ET, 4:25PM ET, 1...",[https://www.pro-football-reference.com/boxsco...,[],"[, , , , , , , , , , , , , , , , ]","[0-1, 1-1, 2-1, 3-1, 3-2, , 3-3, 3-4, 3-5, 4-5...","[, @, @, , @, , , , @, , @, , @, , @, , @]","[Green Bay Packers, Denver Broncos, Washington...",...,[],[],[],[],[],[],[],[],[],NaN
cin 2019,[],"[Sun, Sun, Sun, Mon, Sun, Sun, Sun, Sun, , Sun...","[September 8, September 15, September 22, Sept...","[4:05PM ET, 1:00PM ET, 1:00PM ET, 8:15PM ET, 1...",[https://www.pro-football-reference.com/boxsco...,[],"[, , , , , , , , , , , , , , , OT, ]","[0-1, 0-2, 0-3, 0-4, 0-5, 0-6, 0-7, 0-8, , 0-9...","[@, , @, @, , @, , @, , , @, , , @, , @, ]","[Seattle Seahawks, San Francisco 49ers, Buffal...",...,[],[],[],[],[],[],[],[],[],NaN
cle 2019,[],"[Sun, Mon, Sun, Sun, Mon, Sun, , Sun, Sun, Sun...","[September 8, September 16, September 22, Sept...","[1:00PM ET, 8:15PM ET, 8:20PM ET, 1:00PM ET, 8...",[https://www.pro-football-reference.com/boxsco...,[],"[, , , , , , , , , , , , , , , , ]","[0-1, 1-1, 1-2, 2-2, 2-3, 2-4, , 2-5, 2-6, 3-6...","[, @, , @, @, , , @, @, , , , @, , @, , @]","[Tennessee Titans, New York Jets, Los Angeles ...",...,[],[],[],[],[],[],[],[],[],NaN
dal 2019,[],"[Sun, Sun, Sun, Sun, Sun, Sun, Sun, , Mon, Sun...","[September 8, September 15, September 22, Sept...","[4:25PM ET, 1:00PM ET, 1:00PM ET, 8:20PM ET, 4...",[https://www.pro-football-reference.com/boxsco...,[],"[, , , , , , , , , , , , , , , , ]","[1-0, 2-0, 3-0, 3-1, 3-2, 3-3, 4-3, , 5-3, 5-

In [295]:
# testing pulling boxscore url link

def boxscore(url):
    table_gamelog = soup.find('div', id = 'div_games')
    for count1, row in enumerate(table_gamelog.find_all('tr')):
        for count, x in enumerate(row.find_all('td')):
            for a in x.find_all('a', href=True):
                a = a['href']
                if a.startswith('/boxscores'):
                    boxscore = a
    box_url = base + boxscore
    boxstats = pullTable(box_url,'team_stats')
    boxstats.columns = ['Stats', 'Home', 'Away']
    homebox = boxscoretest[['Stats','Home']]
    awaybox = boxscoretest[['Stats','Away']]
    homebox = homebox.transpose()
    awaybox = awaybox.transpose()
    home_colnames = ['Home_' + x for x in homebox.iloc[0]]
    away_colnames = ['Away_' + x for x in awaybox.iloc[0]]
    zip_Homeboxscore = list(zip(home_colnames, homebox.iloc[1]))
    zip_Awayboxscore = list(zip(away_colnames, awaybox.iloc[1]))
    boxscorelist = zip_Homeboxscore + zip_Awayboxscore
    boxscorelist
    return boxscorelist

# boxscoretest = boxscore('https://www.pro-football-reference.com/boxscores/201909150was.htm')
# boxscoretest.columns = ['Stats', 'Home', 'Away']
# homebox = boxscoretest[['Stats','Home']]
# awaybox = boxscoretest[['Stats','Away']]
# homebox = homebox.transpose()
# awaybox = awaybox.transpose()
# home_colnames = ['Home_' + x for x in homebox.iloc[0]]
# away_colnames = ['Away_' + x for x in awaybox.iloc[0]]
# zip_Homeboxscore = list(zip(home_colnames, homebox.iloc[1]))
# zip_Awayboxscore = list(zip(away_colnames, awaybox.iloc[1]))
# boxscorelist = zip_Homeboxscore + zip_Awayboxscore
# boxscorelist




# boxscore2 = boxscore.transpose()
# boxscore.columns


In [296]:
boxscore('https://www.pro-football-reference.com/boxscores/201909150was.htm')

[('Home_First Downs', '13'),
 ('Home_Rush-Yds-TDs', '20-88-0'),
 ('Home_Cmp-Att-Yd-TD-INT', '18-37-206-1-1'),
 ('Home_Sacked-Yards', '2-23'),
 ('Home_Net Pass Yards', '183'),
 ('Home_Total Yards', '271'),
 ('Home_Fumbles-Lost', '1-1'),
 ('Home_Turnovers', '2'),
 ('Home_Penalties-Yards', '2-10'),
 ('Home_Third Down Conv.', '1-12'),
 ('Home_Fourth Down Conv.', '0-3'),
 ('Home_Time of Possession', '25:45'),
 ('Away_First Downs', '22'),
 ('Away_Rush-Yds-TDs', '36-223-1'),
 ('Away_Cmp-Att-Yd-TD-INT', '23-33-303-4-0'),
 ('Away_Sacked-Yards', '3-9'),
 ('Away_Net Pass Yards', '294'),
 ('Away_Total Yards', '517'),
 ('Away_Fumbles-Lost', '1-1'),
 ('Away_Turnovers', '1'),
 ('Away_Penalties-Yards', '10-81'),
 ('Away_Third Down Conv.', '8-15'),
 ('Away_Fourth Down Conv.', '1-1'),
 ('Away_Time of Possession', '34:15')]

In [ ]:
def boxscoreTranspose(Table):
    Table = Table.transpose()

In [293]:
for count, key in enumerate(ordered_keys)

0 Skip
1 Skip
2 Skip
3 Skip
4 
5 Home
6 Opp
7 1stD
8 TotYd
9 PassY
10 RushY
11 TO
12 1stD
13 TotYd
14 PassY
15 RushY
16 TO
17 Skip
18 Skip
19 Skip
20 Skip
21 Skip
22 Skip
23 Skip
24 Skip
25 Skip
26 Skip
27 Skip
28 Skip
29 Skip
30 Skip
31 Skip
32 Skip
33 Skip


In [ ]:
    for count, row in enumerate(table_gamelog.find_all('tr')):
#         row_data = row.find_all('td')
#         print(len(row_data), row_data)
        if(count - 1) <= max_week and row.text.split('\n')[1].isdigit() and row.text.split('\n')[ordered_key.index('Opp') + 1] != 'Bye Week':
            for count, col in enumerate(row.find_all('td')):
                if count == 0:
                    if col.text == '':
                        dict1[ordered_keys[count]].append(0)
                        week = int(col.text)
                    else:
                        week = int(col.text)
                        dict1[ordered_keys[count]].append(int(col.text))
                elif count in [4,6]:
                    dict1[ordered_key[count]].append(col.text)
                elif count == 8:
                    opp = col.text
                    dict1[ordered_keys[count]].append(col.text)
                elif count == 2:
                    if 'January' in col.text or 'February' in col.text:
                        date_str = get_date(str(int(year)+ 1), col.text)
    

In [2]:
findTables("https://www.pro-football-reference.com/")

['AFC', 'NFC']

In [64]:
AFC = pullTable("https://www.pro-football-reference.com/",'AFC')
NFC = pullTable("https://www.pro-football-reference.com/",'NFC')

In [65]:
AFC = AFC[~AFC.Tm.str.startswith(' AF')]
NFC = NFC[~NFC.Tm.str.startswith(' NF')]

In [66]:
AFC['Tm'] = AFC['Tm'].str.replace('*', '', regex=True).str.replace('+', '', regex=True)
NFC['Tm'] = NFC['Tm'].str.replace('*', '', regex=True).str.replace('+', '', regex=True)

In [67]:
NFC

,Tm,W,L,T,W-L%
1,PHI,9,7,0,.563
2,DAL,8,8,0,.500
3,NYG,4,12,0,.250
4,WAS,3,13,0,.188
6,GNB,13,3,0,.813
7,MIN,10,6,0,.625
8,CHI,8,8,0,.500
9,DET,3,12,1,.219
11,NOR,13,3,0,.813
12,ATL,7,9,0,.438


In [ ]:
url = 'https://www.pro-football-reference.com/'     
html = requests.get(url).text                                 
soup = BeautifulSoup(html)